# Conditional Generative Adversarial Network for gene expression inference

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from keras.models import Model
from keras.utils import Sequence
from keras.optimizers import SGD
from keras.metrics import mean_absolute_error
from keras.layers import Input, Dense
from keras.regularizers import l1

Using TensorFlow backend.


# Data Preperation

Download the dataset from [here](https://drive.google.com/file/d/18Et3ewt471dN78tZxDHg7g-a8WI9z00H/view?usp=sharing).

### Load data to Google Colab notebook

In [3]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('')
try:
  os.makedirs(local_download_path)
except: pass

# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '18Et3ewt471dN78tZxDHg7g-a8WI9z00H'
downloaded = drive.CreateFile({'id': file_id})
fname = os.path.join(local_download_path, downloaded['title'])
# print('Downloaded content "{}"'.format(downloaded.GetContentString()))

print('title: %s, id: %s' % (downloaded['title'], downloaded['id']))
downloaded.GetContentFile(fname)
print(fname)

     |████████████████████████████████| 993kB 5.0MB/s 
title: HW6-Data.gctx, id: 18Et3ewt471dN78tZxDHg7g-a8WI9z00H
HW6-Data.gctx


### Read .gctx data

In [4]:
!pip install -q cmapPy
# https://github.com/cmap/cmapPy/blob/master/tutorials/cmapPy_pandasGEXpress_tutorial.ipynb

     |████████████████████████████████| 153kB 5.1MB/s 


In [0]:
# Load and reshape data so each row corresponds to a sample
from cmapPy.pandasGEXpress.parse import parse
gctx_data = parse("HW6-Data.gctx")

In [6]:
data_df = gctx_data.data_df.T
print(data_df.shape)
data_df.head()

(2921, 5503)


rid,ENSG00000175063.12,ENSG00000171174.9,ENSG00000160326.9,ENSG00000204209.6,ENSG00000087460.18,ENSG00000105968.14,ENSG00000163686.9,ENSG00000079739.11,ENSG00000134057.10,ENSG00000196154.7,ENSG00000148400.9,ENSG00000164362.14,ENSG00000176171.7,ENSG00000119185.8,ENSG00000115738.5,ENSG00000171453.13,ENSG00000214063.6,ENSG00000147383.6,ENSG00000103876.7,ENSG00000109654.10,ENSG00000137843.7,ENSG00000104365.9,ENSG00000123130.12,ENSG00000100225.13,ENSG00000010810.12,ENSG00000126602.6,ENSG00000152601.13,ENSG00000149658.13,ENSG00000169598.11,ENSG00000087088.15,ENSG00000095066.7,ENSG00000161204.7,ENSG00000198369.5,ENSG00000145293.10,ENSG00000065534.14,ENSG00000072062.9,ENSG00000165704.10,ENSG00000170502.8,ENSG00000105699.12,ENSG00000172175.8,...,ENSG00000125753.9,ENSG00000125741.4,ENSG00000064692.14,ENSG00000010310.4,ENSG00000125743.6,ENSG00000011478.7,ENSG00000177051.5,ENSG00000177045.6,ENSG00000064652.6,ENSG00000170604.3,ENSG00000151292.13,ENSG00000064199.2,ENSG00000154146.7,ENSG00000149548.10,ENSG00000182013.13,ENSG00000160013.4,ENSG00000134910.8,ENSG00000155324.5,ENSG00000090372.10,ENSG00000181027.6,ENSG00000105281.8,ENSG00000110060.4,ENSG00000113368.7,ENSG00000173926.5,ENSG00000130748.6,ENSG00000064309.10,ENSG00000130749.5,ENSG00000142230.7,ENSG00000105327.11,ENSG00000105321.8,ENSG00000064651.9,ENSG00000110063.4,ENSG00000063169.6,ENSG00000105499.9,ENSG00000142227.6,ENSG00000161558.6,ENSG00000043039.5,ENSG00000105438.4,ENSG00000105464.3,ENSG00000182324.5
cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GTEX-N7MS-0007-SM-2D7W1,6.258000,1.570040,12.819815,25.197998,114.751381,19.435654,1.811746,13.255496,3.265682,272.174011,18.312820,0.342227,7.193198,6.058734,35.885273,5.476271,4.680382,5.160254,5.301565,0.404902,0.447544,19.715021,12.691573,363.121033,35.115589,10.643700,18.738678,12.508470,3.961152,44.450912,5.175422,19.583858,1.780967,7.288062,1.285417,38.469940,8.974514,4.648564,8.893391,9.216136,...,127.530022,2.516836,0.008248,4.131555,20.181681,9.524962,9.139047,0.583649,1.100281,7.937450,2.745857,1.039201,60.472664,1.185207,0.084997,5.044806,14.393388,2.090897,23.079409,4.407234,19.984457,4.492527,17.262720,3.341017,5.359549,0.092620,6.270971,17.616255,13.506384,26.032234,1.507246,16.141371,8.745433,0.626008,143.632782,4.309718,0.097738,37.715435,0.264591,0.008248
GTEX-N7MS-0008-SM-4E3JI,38.256783,1.417273,3.846615,13.019768,65.975456,21.775644,2.026841,21.197948,56.756538,343.272125,2.461825,0.007284,60.472664,18.912960,29.144449,13.602143,34.434437,11.517919,6.471162,2.251328,0.007284,7.389894,12.720635,28.596855,22.336266,16.432726,28.066408,13.506384,2.540541,54.584293,3.453465,15.831875,6.571239,19.810352,3.938659,25.686693,28.365864,6.580140,1.645448,6.144692,...,38.674606,2.775685,0.404902,0.950917,90.587181,6.771498,3.824857,4.283224,4.111968,8.198557,6.310730,9.505074,5.807346,1.890565,1.013015,2.915665,47.711220,11.408774,15.766845,5.973472,67.406731,4.293163,25.409655,1.880815,15.609934,0.660897,6.270971,47.598354,2.918443,5.741512,3.179466,15.544182,3.222655,0.492695,139.820328,2.678623,0.007284,113.112526,2.248742,0.587866
GTEX-N7MS-0011-R10A-SM-2HMJK,0.018755,0.958147,9.762014,11.459002,227.624619,35.348461,8.654851,13.895129,3.640244,2.266490,2.868886,0.046608,37.521179,21.958830,46.217178,8.708959,2.130838,7.838990,1.977618,25.197998,3.091491,5.194419,3.725604,18.756191,25.409655,16.717360,9.929487,14.231618,2.824677,9.293390,7.752186,22.701349,13.101074,43.441166,3.741034,39.985050,60.657154,12.527876,3.945090,1.513837,...,4.363213,3.400148,0.699935,0.137225,49.061504,6.991272,3.069155,0.510642,5.990269,17.886543,5.495917,5.217256,977.106323,0.128216,58.799564,0.174970,6.122733,7.323215,24.217220,6.058734,0.293274,2.689233,1.145243,0.296909,38.189690,1.137451,5.081839,33.039825,2.716285,5.587399,5.591400,6.231666,3.787384,7.064671,5.623240,3.376416,0.472907,20.261278,2.430775,0.449920
GTEX-N7MS-0011-R11A-SM-2HMJS,0.579502,1.871301,4.256632,15.027001,256.823669,64.796654,10

### Prepare data for training

In [0]:
# Split data of landmark and target genes so each 
data = data_df.to_numpy()
n_landmark = 943  # number of landmark genes
n_est = np.size(data, axis=1) - n_landmark  # number of genes to be estimated
X = data[:, :n_landmark]
Y = data[:, n_landmark:]

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [0]:
# Normalize data using train mean and std for each gene
x_train_mean = x_train.mean(axis=0)
x_train_std = x_train.std(axis=0)
y_train_mean = y_train.mean(axis=0)
y_train_std = y_train.std(axis=0)
# print("Mean: ", train_mean)
# print("Std: ", train_std)

# normalization
def normalize(x, mean, std):
  return (x - mean) / std

x_train = normalize(x_train, x_train_mean, x_train_std)
x_test = normalize(x_test, x_train_mean, x_train_std)
y_train = normalize(y_train, y_train_mean, y_train_std)
y_test = normalize(y_test, y_train_mean, y_train_std)

# Base Model

In [10]:
def build_predictor():
    input_data = Input(shape=(n_landmark,))
    h = Dense(3000, activation='linear')(input_data)
    out = Dense(n_est, activation='linear')(h)
    model = Model(input_data, out, name='generator')
    return model

base_model = build_predictor()
base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 943)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3000)              2832000   
_________________________________________________________________
dense_2 (Dense)              (None, 4560)              13684560  
Total params: 16,516,560
Trainable params: 16,516,560
Non-trainable params: 0
_________________________________________________________________


In [11]:
base_model.compile(loss='mean_absolute_error', optimizer=SGD(lr=0.1))
history = base_model.fit(x_train, y_train, 
                         epochs=100, batch_size=16, 
                         validation_data=(x_test, y_test), verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 2190 samples, validate on 731 samples
Epoch 1/100
2190/2190 [==============================] - 4s 2ms/step - loss: 0.7253 - val_loss: 0.6644
Epoch 2/100
2190/2190 [==============================] - 1s 473us/step - loss: 0.6192 - val_loss: 0.5848
Epoch 3/100
2190/2190 [==============================] - 1s 466us/step - loss: 0.5516 - val_loss: 0.5307
Epoch 4/100
2190/2190 [==============================] - 1s 464us/step - loss: 0.5053 - val_loss: 0.4936
Epoch 5/100
2190/2190 [==============================] - 1s 475us/step - loss: 0.4729 - val_loss: 0.4669
Epoch 6/100
2190/2190 [==============================] - 1s 470us/step - loss: 0.4490 - val_loss: 0.4467
Epoch 7/100
2190/2190 [==============================] - 1s 465us/step - loss: 0.4301 - val_loss: 0.4303
Epoch 8/100
2190/2190 [==============================] - 1s 470us/step - loss: 0.4146 - val_loss: 0.4167
Epoch 9/100
2190/2190 [==============================] - 1s 468us/s

In [12]:
# report MAE on train and test data
print("Train MAE: ", base_model.evaluate(x_train, y_train))
print("Test MAE: ", base_model.evaluate(x_test, y_test))

2190/2190 [==============================] - 0s 52us/step
Train MAE:  0.22059063235102178
731/731 [==============================] - 0s 48us/step
Test MAE:  0.25253316225985983


# Adversarial Model

In [0]:
# parameters
g_lr = 0.05
d_lr = 0.1
reg_param = 0.01

In [0]:
def build_discriminator():
    input_data = Input(shape=(n_est,))
    hidden_layer = Dense(700, activation='linear', kernel_regularizer=l1(reg_param))(input_data) 
#                          bias_regularizer=l2(0.01))
    output = Dense(1, activation='sigmoid')(hidden_layer)
    model = Model(input_data, output, name='discriminator')
    return model

In [15]:
discriminator = build_discriminator()
discriminator_optimizer = SGD(lr=d_lr)
print(discriminator.summary())

discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 4560)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 700)               3192700   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 701       
Total params: 3,193,401
Trainable params: 3,193,401
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
generator = build_predictor()

# Freeze discriminator weights using `trainable` parameter
discriminator.trainable = False

# then create the combined model using generator and discriminator models.
gan_in = Input(shape=(n_landmark,), name='input')
y_gen = generator(gan_in)
valid = discriminator(y_gen)

gan_model = Model(input=gan_in, output=[y_gen, valid], name='gan')
print(gan_model.summary())
# Finally compile the combined model
gan_optimizer = SGD(lr=g_lr)
gan_model.compile(loss={'generator': 'mean_absolute_error', 'discriminator': 'binary_crossentropy'}, 
                  optimizer=gan_optimizer, 
                  metrics={'discriminator': 'accuracy'})

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 943)               0         
_________________________________________________________________
generator (Model)            (None, 4560)              16516560  
_________________________________________________________________
discriminator (Model)        (None, 1)                 3193401   
Total params: 19,709,961
Trainable params: 16,516,560
Non-trainable params: 3,193,401
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="gan", inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  # This is added back by InteractiveShellApp.init_path()


In [0]:
class DataGenerator(Sequence):
    """
        Generates batches of data. Useful for training manually in Keras.
        If necessary, you may update this class to fit your needs.
        Adopted from: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
    """
    def __init__(self, train_X, train_y, batch_size, shuffle=True):
        self.batch_size = batch_size
        self.train_X = train_X
        self.train_y = train_y
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.train_X) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        return self.train_X[indexes], self.train_y[indexes]


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.train_X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


In [0]:
batch_size = 16
epochs = 500

In [0]:
data_generator = DataGenerator(x_train, y_train, batch_size=16)

In [20]:
print('GAN metrics: ', gan_model.metrics_names)
print('discriminator metrics: ', discriminator.metrics_names)

GAN metrics:  ['loss', 'generator_loss', 'discriminator_loss', 'discriminator_acc']
discriminator metrics:  ['loss', 'acc']


In [21]:
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):
    
    n = 0
    g_total_loss = 0
    g_total_mae = 0
    d_total_loss = 0
    d_total_accuracy = 0
    
    for X, y in data_generator:
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        
        # Train the discriminator and calculate loss and accuracy
        y_fake = generator.predict(X)
        
        # then use discriminator train_on_batch twice.
        # once with y as input and once with the output of generator.predict as input.
        d_loss_real = discriminator.train_on_batch(y, valid)
        d_loss_fake = discriminator.train_on_batch(y_fake, fake)
        
        d_loss = d_loss_real[0] + d_loss_fake[0]
        d_accuracy = d_loss_real[1] + d_loss_fake[1]

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator and calculate loss and MAE
        g_losses = gan_model.train_on_batch(X, [y, valid])
        
        g_loss = g_losses[0]
        g_mae = g_losses[2]
        
        d_total_loss = (d_loss + d_total_loss) / 2
        d_total_accuracy = (d_accuracy + d_total_accuracy) / 2
        g_total_loss = g_loss + g_total_loss
        g_total_mae = g_mae + g_total_mae
        n += 1
        
#         print("%d Train: [D loss: %f, acc.: %.2f%%] [G loss: %f G MAE: %f]" % 
#               (epoch, d_total_loss / n, d_total_accuracy / n, g_total_loss / n, g_total_mae / n))
        
    y_test_fake = generator.predict(x_test)
    test_score_real = gan_model.evaluate(x_test, [y_test, np.ones((y_test.shape[0], 1))], verbose=0)
    test_score_fake = gan_model.evaluate(x_test, [y_test_fake, np.zeros((y_test.shape[0], 1))], verbose=0)
    test_score = np.add(test_score_real, test_score_fake) / 2
    
    test_d_loss = test_score[2]
    test_d_accuracy = test_score[3]
    test_g_loss = test_score[0]
    test_g_mae = test_score_real[1]
    
    
    print ("%d Train: [D loss: %f, acc.: %.2f%%] [G loss: %f G MAE: %f]" % 
           (epoch, d_total_loss / n, d_total_accuracy * 100 / n, g_total_loss / n, g_total_mae / n))
    print ("%d Test: [D loss: %f, acc.: %.2f%%] [G loss: %f G MAE: %f]" % 
           (epoch, test_d_loss, test_d_accuracy * 100, test_g_loss, test_g_mae))
    print('=================================================================================')
    
    data_generator.on_epoch_end()


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 Train: [D loss: 0.245199, acc.: 0.00%] [G loss: 37.272769 G MAE: 1.024674]
0 Test: [D loss: 0.693236, acc.: 50.00%] [G loss: 16.952491 G MAE: 0.703014]
1 Train: [D loss: 0.245162, acc.: 0.00%] [G loss: 17.270460 G MAE: 0.708476]
1 Test: [D loss: 0.693236, acc.: 50.00%] [G loss: 16.888998 G MAE: 0.650990]
2 Train: [D loss: 0.245132, acc.: 0.00%] [G loss: 17.188124 G MAE: 0.707990]
2 Test: [D loss: 0.693234, acc.: 50.00%] [G loss: 16.844282 G MAE: 0.610117]
3 Train: [D loss: 0.245114, acc.: 0.00%] [G loss: 17.127031 G MAE: 0.707596]
3 Test: [D loss: 0.693236, acc.: 50.00%] [G loss: 16.798920 G MAE: 0.576442]
4 Train: [D loss: 0.245110, acc.: 0.00%] [G loss: 17.061786 G MAE: 0.707177]
4 Test: [D loss: 0.693236, acc.: 50.00%] [G loss: 16.750573 G MAE: 0.548374]
5 Train: [D loss: 0.245115, acc.: 0.00%] [G loss: 17.004796 G MAE: 0.707184]
5 Test: [D loss: 0.693236, acc.: 50.00%] [G loss: 16.706819 G MAE: 0.525019]
6 Train: [D loss: 0.245108, acc.: 0.00%] [G loss: 16.947038 G MAE: 0.707197]

In [22]:
# report MAE on train and test data

test_score = gan_model.evaluate(x_test, [y_test, np.ones((y_test.shape[0], 1))])
train_score = gan_model.evaluate(x_train, [y_train, np.ones((y_train.shape[0], 1))])

print ("Train MAE: %f" % test_score[1])
print ("Test MAE: %f" % train_score[1])

2190/2190 [==============================] - 0s 68us/step
Train MAE: 0.224171
Test MAE: 0.180115
